<a href="https://colab.research.google.com/github/DubblePos/Colab/blob/main/Ch04/2_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
날짜 : 2021/11/23
이름 : 박승필
내용 : 영화 리뷰 감성분석 실습하기
"""
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 23.4 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pickle, re
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Activation, Dropout, Embedding, LSTM
from keras.preprocessing.sequence import pad_sequences

In [4]:
# 데이터 불러오기
train_data = pd.read_table('/content/drive/MyDrive/Colab Notebooks/file/movie_review_train.txt')
test_data = pd.read_table('/content/drive/MyDrive/Colab Notebooks/file/movie_review_test.txt')
train_data

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [5]:
# 전처리 함수 정의 
okt = Okt()
stop_words = ['은','는','을','를','이','가','하','것','들','의','에게','에서','있','되','등','한']
def preprocessing(text):
  
  # 한글 텍스트 아닌것 제외 
  
  review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',str(text))
  
  # 문장 형태소 분석
  
  review_text = okt.morphs(review_text, stem=True)
  
  # 불용어 제거 
  
  word_review = [w for w in review_text if not w in stop_words]
  
  return word_review
clean_train_review = []
clean_test_review = []

# 훈련 데이터 전처리 
for review in train_data['document']:
  clean_train_review.append(preprocessing(review))
for review in test_data['document']:
  clean_test_review.append(preprocessing(review))
clean_train_review[:5]

[['아', '더빙', '진짜', '짜증나다', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'],
 ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다'],
 ['교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정'],
 ['사이',
  '몬페',
  '그',
  '익살스럽다',
  '연기',
  '돋보이다',
  '영화',
  '스파이더맨',
  '늙다',
  '보이다',
  '하다',
  '커스틴',
  '던스트',
  '너무나도',
  '이쁘다',
  '보이다']]

In [6]:
# 토큰화 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_review)
train_sequences = tokenizer.texts_to_sequences(clean_train_review)
test_sequences = tokenizer.texts_to_sequences(clean_test_review)
word_dic = tokenizer.word_index
word_dic

{'영화': 1,
 '보다': 2,
 '하다': 3,
 '에': 4,
 '도': 5,
 '없다': 6,
 '이다': 7,
 '있다': 8,
 '좋다': 9,
 '너무': 10,
 '다': 11,
 '정말': 12,
 '재밌다': 13,
 '되다': 14,
 '적': 15,
 '만': 16,
 '같다': 17,
 '진짜': 18,
 '으로': 19,
 '로': 20,
 '아니다': 21,
 '않다': 22,
 '점': 23,
 '만들다': 24,
 '과': 25,
 '나오다': 26,
 '연기': 27,
 '평점': 28,
 '최고': 29,
 '내': 30,
 '그': 31,
 '나': 32,
 '안': 33,
 '인': 34,
 '스토리': 35,
 '생각': 36,
 '못': 37,
 '왜': 38,
 '드라마': 39,
 '게': 40,
 '감동': 41,
 '사람': 42,
 '보고': 43,
 '이렇다': 44,
 '말': 45,
 '아깝다': 46,
 '고': 47,
 '더': 48,
 'ㅋㅋ': 49,
 '배우': 50,
 '때': 51,
 '와': 52,
 '아': 53,
 '재미있다': 54,
 '감독': 55,
 '거': 56,
 '그냥': 57,
 '요': 58,
 '재미': 59,
 '시간': 60,
 '내용': 61,
 '뭐': 62,
 '까지': 63,
 '중': 64,
 '주다': 65,
 '재미없다': 66,
 '자다': 67,
 '좀': 68,
 '쓰레기': 69,
 '하고': 70,
 '지루하다': 71,
 '네': 72,
 '수': 73,
 '모르다': 74,
 '가다': 75,
 '들다': 76,
 '그렇다': 77,
 '싶다': 78,
 '지': 79,
 '작품': 80,
 '사랑': 81,
 '알다': 82,
 '하나': 83,
 '다시': 84,
 '볼': 85,
 '마지막': 86,
 '잘': 87,
 '이건': 88,
 '정도': 89,
 '저': 90,
 '완전': 91,
 'ㅋ': 92,
 '오다': 93,
 '

In [7]:
# 패딩
train_inputs = pad_sequences(train_sequences,maxlen=10,padding='post') # 텍스트 10개짜리 패딩을 걸어서 학습시킴
test_inputs = pad_sequences(test_sequences, maxlen=10,padding='post')
train_labels = np.array(train_data['label'])
test_labels = np.array(test_data['label'])

print(train_inputs[0])
print(train_labels[0])

[ 53 462  18 263 664   0   0   0   0   0]
0


In [8]:
# 모델생성
model = Sequential()
model.add(Embedding(30000,10)) # 30000개의 단어를 입력할 수 있는 층  10개의 노드로 시작 
model.add(LSTM(128))           # 순환신경망 - RNN모델에서 좀더 장기기억을 위한 순환 엣지를 하나 더 추가한 LSTM
model.add(Dense(1,activation='sigmoid')) # 출력노드는 하나이다. 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 10)          300000    
                                                                 
 lstm (LSTM)                 (None, 128)               71168     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 371,297
Trainable params: 371,297
Non-trainable params: 0
_________________________________________________________________


In [9]:
# 모델설정

model.compile(loss='binary_crossentropy',
               optimizer='adam',
               metrics=['acc'])

In [10]:
# 모델학습

model.fit(train_inputs,
          train_labels,
          epochs=10,
          batch_size=128,
          validation_split=0.2)

Epoch 1/10
938/938 [==============================] - 18s 11ms/step - loss: 0.4391 - acc: 0.7893 - val_loss: 0.3864 - val_acc: 0.8270
Epoch 2/10
938/938 [==============================] - 9s 9ms/step - loss: 0.3489 - acc: 0.8468 - val_loss: 0.3866 - val_acc: 0.8280
Epoch 3/10
938/938 [==============================] - 9s 9ms/step - loss: 0.3195 - acc: 0.8619 - val_loss: 0.3916 - val_acc: 0.8270
Epoch 4/10
938/938 [==============================] - 9s 10ms/step - loss: 0.2946 - acc: 0.8739 - val_loss: 0.3932 - val_acc: 0.8285
Epoch 5/10
938/938 [==============================] - 9s 9ms/step - loss: 0.2693 - acc: 0.8861 - val_loss: 0.4117 - val_acc: 0.8245
Epoch 6/10
938/938 [==============================] - 9s 10ms/step - loss: 0.2462 - acc: 0.8968 - val_loss: 0.4378 - val_acc: 0.8232
Epoch 7/10
938/938 [==============================] - 9s 9ms/step - loss: 0.2255 - acc: 0.9059 - val_loss: 0.4944 - val_acc: 0.8210
Epoch 8/10
938/938 [==============================] - 9s 9ms/step - loss

In [11]:
# 모델평가
result = model.evaluate(test_inputs,test_labels)
result

1563/1563 [==============================] - 6s 4ms/step - loss: 0.6026 - acc: 0.8073


[0.6026345491409302, 0.8072999715805054]

In [12]:
#
def sentiment_predict(text):
  
  # 한글 텍스트 아닌것 제외 
  
  review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',str(text))
  
  # 문장 형태소 분석
  
  review_text = okt.morphs(review_text, stem=True)
  
  # 불용어 제거 
  
  word_review = [w for w in review_text if not w in stop_words]
  
  encoded = tokenizer.texts_to_sequences([word_review])
  padded = pad_sequences(encoded, maxlen=10, padding='post')
  score = float(model.predict(padded))
  return score

In [ ]:
while True:
  review = input('리뷰 입력: ')
  score = sentiment_predict(review)
  if score > 0.5:
    print('{:.2f}% 확률로 긍정 리뷰입니다.\n'.format(score*100))
  else:
    print('{:.2f}% 확률로 부정 리뷰입니다.\n'.format((1-score)*100))

리뷰 입력: 정말 재미없어요!
99.62% 확률로 부정 리뷰입니다.

리뷰 입력: 짱이에요!
99.33% 확률로 긍정 리뷰입니다.

